#### With pydeck, users can embed visualizations interactively in a Jupyter Notebook or simply export them to a standalone HTML file.

https://www.mapbox.com/

In [1]:
import os
from dotenv import load_dotenv
# .env file to environment
#load_dotenv()
load_dotenv(verbose=True)

token = os.getenv('MAPBOX_API_TOKEN')

In [8]:
import mapboxgl
import pydeck as pdk
import pandas as pd
import numpy as np

In [ ]:
# 2014 locations of car accidents in the UK
UK_ACCIDENTS_DATA = ('https://raw.githubusercontent.com/uber-common/'
                     'deck.gl-data/master/examples/3d-heatmap/heatmap-data.csv')

In [16]:
pd.read_csv(UK_ACCIDENTS_DATA)

,lng,lat
0,-0.198465,51.505538
1,-0.178838,51.491836
2,-0.205590,51.514910
3,-0.208327,51.514952
4,-0.206022,51.496572
...,...,...
140051,-3.376671,55.023855
140052,-3.242159,55.016316
140053,-3.387067,55.163502
140054,-3.123385,55.020580


In [35]:
# Define a layer to display on a map
layer = pdk.Layer(
    'HexagonLayer',
    UK_ACCIDENTS_DATA,
    get_position=['lng', 'lat'],
    auto_highlight=True,
    elevation_scale=50,
    pickable=True,
    elevation_range=[0, 3000],
    extruded=True,                 
    coverage=1)

# Set the viewport location
view_state = pdk.ViewState(
    longitude=-1.415,
    latitude=52.2323,
    zoom=6,
    min_zoom=5,
    max_zoom=15,
    pitch=40.5,
    bearing=-27.36)

# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html('demo.html')

r.show()

#### Update afterwards

In [36]:
layer.elevation_range = [0, 10000]
r.update()

In [37]:
r.show()

DeckGLWidget(google_maps_key=None, json_input='{"initialViewState": {"bearing": -27.36, "latitude": 52.2323, "…

#### Update visualization over time

In [41]:
import time
r.show()

for i in range(0, 10000, 1000):
    layer.elevation_range = [0, i]
    r.update()
    time.sleep(0.1)

In [43]:
 # Combined all of it and render a viewport
r = pdk.Deck(
      layers=[layer],
      initial_view_state=view_state,
      tooltip={
          'html': '<b>Elevation Value:</b> {elevationValue}',
          'style': {
              'color': 'white'
          }
      }
  )
r.show()

DeckGLWidget(google_maps_key=None, json_input='{"initialViewState": {"bearing": -27.36, "latitude": 52.2323, "…

In [44]:
# Combined all of it and render a viewport
r = pdk.Deck(
      layers=[layer],
      initial_view_state=view_state,
      tooltip={
          "text": "Elevation: {elevationValue}"
      }
  )
r.show()

DeckGLWidget(google_maps_key=None, json_input='{"initialViewState": {"bearing": -27.36, "latitude": 52.2323, "…

In [ ]:
LIGHTS_URL = 'https://raw.githubusercontent.com/ajduberstein/lights_at_night/master/chengdu_lights_at_night.csv'
df = pd.read_csv(LIGHTS_URL)

In [47]:
df.head()

,year,lng,lat,brightness,color
0,1993,104.575,31.808,4,"[255, 16, 255, 255]"
1,1993,104.583,31.808,4,"[255, 16, 255, 255]"
2,1993,104.592,31.808,4,"[255, 16, 255, 255]"
3,1993,104.600,31.808,4,"[255, 16, 255, 255]"
4,1993,104.675,31.808,4,"[255, 16, 255, 255]"


In [48]:
df.brightness.unique()

array([ 4,  5,  6,  7, 15, 17, 18, 12,  8, 13, 16, 21, 22, 19, 11,  3, 14,
       23, 27, 28, 26, 24,  9, 25, 29, 33, 36, 35, 34, 38, 39, 37, 31, 41,
       32, 42, 43, 44, 45, 46, 47, 48, 49, 51, 52, 54, 53, 55, 57, 56, 58,
       59, 61, 62, 63,  2])

In [50]:
df.year.unique()

array([1993, 1997, 1995, 2009, 2001, 2003, 2013, 2007, 2011, 2005, 1999])

In [45]:
df['color'] = df['brightness'].apply(lambda val: [255, val * 4,  255, 255])
plottable = df[df['year'] == 1993].to_dict(orient='records')


view_state = pdk.ViewState(
      latitude=31.0,
      longitude=104.5,
      zoom=8,
      max_zoom=8,
      min_zoom=8)

scatterplot = pdk.Layer(
      'HeatmapLayer',
      data=plottable,
      get_position='[lng, lat]',
      get_weight='brightness',
      opacity=0.5,
      pickable=False,
      get_radius=800)

r = pdk.Deck(
      layers=[scatterplot],
      initial_view_state=view_state,
      views=[pdk.View(type='MapView', controller=None)])
r.show()

DeckGLWidget(google_maps_key=None, json_input='{"initialViewState": {"latitude": 31.0, "longitude": 104.5, "ma…

In [52]:
# Widget slider
import ipywidgets as widgets
from IPython.display import display
slider = widgets.IntSlider(1992, min=1993, max=2013, step=2)

# Define function to update deck 
def on_change(v):
      results = df[df['year'] == slider.value].to_dict(orient='records')
      scatterplot.data = results
      r.update()
   
#

#receives value from the slider and call function to update deck
slider.observe(on_change, names='value')
display(slider)

IntSlider(value=1993, max=2013, min=1993, step=2)